In [ ]:
# ==============================================================================
# SCRIPT: Extract_ViT_Features_With_Path_and_Labels.py
# DESCRIPTION: This script extracts Vision Transformer (ViT) features from images 
#              in a dataset organized by hospital classes (C_0 to C_4) with subfolders
#              (L_0 or L_1). It saves feature vectors along with full file paths, 
#              dataset labels (C_0 to C_4), and inner labels (L_0 or L_1) in .pkl files
#              for each hospital. The script uses a pretrained ViT model from Hugging Face.
# AUTHOR: Amir Soleimani-Yazdi
# DATE: 2025-09-19
# ==============================================================================

# --- 1. Imports ---
# ------------------------------------------------------------------------------
import os
import torch
from transformers import AutoImageProcessor, ViTModel
from PIL import Image
import numpy as np
from tqdm import tqdm
import pickle

# Other imports for potential downstream analysis (kept for compatibility)
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from scipy.spatial.distance import pdist, squareform
import matplotlib.pyplot as plt
import seaborn as sns
import random

# --- 2. Load Pretrained ViT Model and Image Processor ---
# ------------------------------------------------------------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define the Hugging Face model name
model_name = "google/vit-base-patch16-224-in21k"

# Load the ViT model and its corresponding image processor
model = ViTModel.from_pretrained(model_name).to(device)
image_processor = AutoImageProcessor.from_pretrained(model_name, use_fast=True)
model.eval()
print(f"✅ ViT model '{model_name}' loaded on {device}.")

# --- 3. Feature Extraction Function for ViT ---
# ------------------------------------------------------------------------------
def extract_features_vit(image_paths, model, processor, hospital_id):
    features = []
    for path in tqdm(image_paths, desc=f"Extracting ViT features for {hospital_id}"):
        try:
            img = Image.open(path).convert('RGB')
            
            # Use the ViT image processor
            inputs = processor(images=img, return_tensors="pt").to(device)

            with torch.no_grad():
                # Get the model's output
                outputs = model(**inputs)
                
                # Use the .pooler_output for the [CLS] token representation (shape: 768,)
                feature = outputs.pooler_output.squeeze().cpu().numpy()
                
                # Extract inner label (L_0 or L_1) from the path
                inner_label = 'L_0' if 'L_0' in path else 'L_1' if 'L_1' in path else 'Unknown'
                
                # Store tuple: (full_path, feature_vector, dataset_label, inner_label)
                features.append((path, feature, hospital_id, inner_label))
                
        except Exception as e:
            print(f"❌ Error with {path}: {e}")
    return features

# --- 4. Extract and Save Features for Each Hospital ---
# ------------------------------------------------------------------------------
dataset_dir = r"d:\new_patches\new_patches"
hospital_features = {}

# Define output directory for .pkl files (same as t-SNE script for compatibility)
output_dir = r"d:\.kaggle\new_FE_ViT_with_Path_and_Labels"
os.makedirs(output_dir, exist_ok=True)

for i in range(5):
    hospital_id = f"C_{i}"
    hospital_path = os.path.join(dataset_dir, hospital_id)

    image_paths = []
    for root, _, files in os.walk(hospital_path):
        image_paths += [os.path.join(root, f) for f in files if f.lower().endswith(('png', 'jpg', 'jpeg'))]

    print(f"\n➡️ {hospital_id} - Total Images: {len(image_paths)}")

    if image_paths:
        # Call the feature extraction function with hospital_id
        feature_vectors = extract_features_vit(image_paths, model, image_processor, hospital_id)
        
        # Save to .pkl file in the output directory
        output_filename = os.path.join(output_dir, f"features_vit_{hospital_id}.pkl")
        with open(output_filename, "wb") as f:
            pickle.dump(feature_vectors, f)
            
        hospital_features[hospital_id] = feature_vectors
        print(f"💾 Saved {output_filename}")

print("\n--- Feature Extraction Complete ---")

In [ ]:
#looading and preparing latent vectors and Tsne coordinations 

# ==============================================================================
# SCRIPT: ViT_Feature_tSNE_Analysis_Full_C2_Distances.py
# DESCRIPTION: This script performs t-Distributed Stochastic Neighbor Embedding (t-SNE)
#              on the full set of Vision Transformer (ViT) feature vectors from all classes
#              (C_0 to C_4). It generates an Excel file with filename, path, t-SNE coordinates,
#              dataset (C_0 to C_4), inner label (L_0 or L_1), and distances to all centroids
#              for all samples. A separate Excel file is generated for C_2 samples' distances
#              to centroids. Visualizations (scatter plot, heatmaps) use subsampled points for
#              efficiency. Outputs are saved in a new folder for full t-SNE results.
# AUTHOR:
# DATE: 2025-09-19
# ==============================================================================

# --- 1. Import Necessary Libraries ---
# ------------------------------------------------------------------------------
import numpy as np
import pickle
import os
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from scipy.spatial.distance import pdist, squareform, cosine, cdist

# --- 2. Configuration and Setup ---
# ------------------------------------------------------------------------------
# Define the base directory where feature files are located.
FEATURE_DIR = r"d:\.kaggle"

# Define the directory where all outputs (plots, data files) will be saved.
OUTPUT_DIR = r"d:\.kaggle\outputs_newRR5_vit_tsne_full09192025"

# Create the output directory if it doesn't already exist.
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Subsampling configuration (for visualization only)
SUBSAMPLE_SIZE = 1000  # Maximum total samples for visualization
SUBSAMPLE_FRACTION = 0.1  # Fraction per class for visualization

print("--- Configuration Complete ---")
print(f"Feature source directory: {FEATURE_DIR}")
print(f"Output directory: {OUTPUT_DIR}\n")

# --- 3. Data Loading and Preprocessing ---
# ------------------------------------------------------------------------------
print("--- Step 1: Loading and Preparing Data ---")

# A dictionary to hold the feature vectors and filenames for each class/hospital.
all_features = {}
all_filenames = {}

# Initialize empty lists to aggregate data from all files.
labels = []
feature_vectors = []
filenames = []

# Loop through the 5 classes (C_0 to C_4).
for i in range(5):
    hospital_id = f"C_{i}"
    
    # Construct the full path to the feature file.
    feature_file_path = os.path.join(FEATURE_DIR, f"features_vit_{hospital_id}.pkl")

    # Check if the file exists before trying to open it.
    if os.path.exists(feature_file_path):
        print(f"--> Loading features for {hospital_id} from {feature_file_path}...")
        try:
            with open(feature_file_path, "rb") as f:
                # Load the data from the pickle file.
                data = pickle.load(f)
                
                # Extract the feature vectors and filenames.
                class_features = [item[1] for item in data]
                class_filenames = [item[0] for item in data]
                
                # Store the loaded features and filenames.
                all_features[hospital_id] = class_features
                all_filenames[hospital_id] = class_filenames
                
                # Append features, filenames, and corresponding labels to master lists.
                feature_vectors.extend(class_features)
                filenames.extend(class_filenames)
                labels.extend([hospital_id] * len(class_features))
                
                print(f"    ✅ Loaded {len(class_features)} feature vectors for {hospital_id}.")
        except Exception as e:
            print(f"    ⚠️ Error loading {feature_file_path}: {e}. Skipping.")
    else:
        print(f"    ⚠️ File not found for {hospital_id} at {feature_file_path}. Skipping.")

# Convert to NumPy arrays.
if not feature_vectors:
    print("\nError: No feature files were found. Exiting script.")
    exit()

feature_vectors = np.array(feature_vectors)
labels = np.array(labels)
filenames = np.array(filenames)
print(f"\nSuccessfully combined all features.")
print(f"Shape of the combined data matrix: {feature_vectors.shape}")

# --- 4. Data Standardization ---
# ------------------------------------------------------------------------------
print("\n--- Step 2: Standardizing Data ---")
scaler = StandardScaler()
standardized_data = scaler.fit_transform(feature_vectors)
print("✅ Full data has been standardized (zero mean, unit variance).\n")

# --- 5. t-SNE Analysis on Full Data ---
# ------------------------------------------------------------------------------
print("--- Step 3: Applying t-SNE to Full Dataset ---")

# Adjust perplexity based on full dataset size (must be less than number of samples).
n_samples = len(feature_vectors)
perplexity = min(30, max(5, n_samples // 3))  # Dynamic perplexity
print(f"Using perplexity={perplexity} for full t-SNE (adjusted for {n_samples} samples).")

# Initialize t-SNE with 2 components for 2D visualization.
tsne = TSNE(n_components=2, perplexity=perplexity, random_state=42, init='pca', n_jobs=-1)
tsne_result = tsne.fit_transform(standardized_data)

print("✅ t-SNE fitting and transformation complete.")
print(f"Shape of data after t-SNE: {tsne_result.shape}")
print(f"KL divergence (cost function): {tsne.kl_divergence_:.4f}")

# --- 6. Subsampling for Visualization ---
# ------------------------------------------------------------------------------
print("\n--- Step 4: Stratified Subsampling for Visualization ---")
sub_indices = []
num_classes = 5  # Assuming 5 classes (C_0 to C_4)
subsample_size_per_class = max(1, SUBSAMPLE_SIZE // num_classes)
unique_labels = sorted(set(labels))

for label in unique_labels:
    label_indices = np.where(labels == label)[0]
    n_label = len(label_indices)
    n_sub_label = min(n_label, max(subsample_size_per_class, int(n_label * SUBSAMPLE_FRACTION)))
    if n_label > 0:
        sub_label_idx = np.random.choice(label_indices, size=n_sub_label, replace=False)
        sub_indices.extend(sub_label_idx)

sub_indices = np.array(sub_indices)
sub_tsne_result = tsne_result[sub_indices]
sub_labels = labels[sub_indices]
sub_filenames = filenames[sub_indices]
print(f"    ✅ Subsampled {len(sub_indices)} points for visualization.")
print(f"    Subsampled t-SNE shape: {sub_tsne_result.shape}")

# --- 7. Calculate Centroids, Euclidean Distances, and Cosine Similarities ---
# ------------------------------------------------------------------------------
print("\n--- Step 5: Calculating Centroids, Euclidean Distances, and Cosine Similarities ---")

# Calculate centroids for each class in t-SNE space (using subsampled data for visualization).
centroids = {}
for label in unique_labels:
    indices = np.where(sub_labels == label)[0]
    centroid = np.mean(sub_tsne_result[indices], axis=0)
    centroids[label] = centroid
    print(f"Centroid for {label}: {centroid}")

# Compute pairwise Euclidean distances between centroids.
centroid_coords = np.array([centroids[label] for label in unique_labels])
euclidean_distances = squareform(pdist(centroid_coords, metric='euclidean'))
euclidean_distance_df = pd.DataFrame(euclidean_distances, index=unique_labels, columns=unique_labels)
print("\nPairwise Euclidean distances between centroids:")
print(euclidean_distance_df)

# Save the Euclidean distance matrix to a CSV file.
euclidean_distance_output_path = os.path.join(OUTPUT_DIR, 'centroid_euclidean_distances.csv')
euclidean_distance_df.to_csv(euclidean_distance_output_path)
print(f"✅ Euclidean centroid distances saved to: {euclidean_distance_output_path}")

# Compute pairwise cosine similarities between centroids.
cosine_similarities = np.zeros((len(unique_labels), len(unique_labels)))
for i, label1 in enumerate(unique_labels):
    for j, label2 in enumerate(unique_labels):
        if i == j:
            cosine_similarities[i, j] = 1.0
        else:
            cosine_similarities[i, j] = 1 - cosine(centroid_coords[i], centroid_coords[j])

cosine_similarity_df = pd.DataFrame(cosine_similarities, index=unique_labels, columns=unique_labels)
print("\nPairwise Cosine similarities between centroids:")
print(cosine_similarity_df)

# Save the cosine similarity matrix to a CSV file.
cosine_similarity_output_path = os.path.join(OUTPUT_DIR, 'centroid_cosine_similarities.csv')
cosine_similarity_df.to_csv(cosine_similarity_output_path)
print(f"✅ Cosine similarities saved to: {cosine_similarity_output_path}")

# --- 8. Generate Excel File with Full t-SNE Results ---
# ------------------------------------------------------------------------------
print("\n--- Step 6: Generating Excel File with Full t-SNE Results ---")

# Create a DataFrame for all samples' t-SNE coordinates and metadata.
excel_data_full = {
    'Filename': [],
    'Path': [],
    'tSNE_X': [],
    'tSNE_Y': [],
    'Dataset': [],
    'Inner_Label': [],
    'Dist_to_C_0': [],
    'Dist_to_C_1': [],
    'Dist_to_C_2': [],
    'Dist_to_C_3': [],
    'Dist_to_C_4': []
}

# Use vectorized distance calculation for all samples to centroids.
all_distances = cdist(tsne_result, centroid_coords, metric='euclidean')

for idx in range(len(filenames)):
    # Extract filename and path.
    file_path = filenames[idx]
    file_name = os.path.basename(file_path)
    
    # Extract inner label (L_0 or L_1) from the path.
    inner_label = 'L_0' if 'L_0' in file_path else 'L_1' if 'L_1' in file_path else 'Unknown'
    
    excel_data_full['Filename'].append(file_name)
    excel_data_full['Path'].append(file_path)
    excel_data_full['tSNE_X'].append(tsne_result[idx, 0])
    excel_data_full['tSNE_Y'].append(tsne_result[idx, 1])
    excel_data_full['Dataset'].append(labels[idx])
    excel_data_full['Inner_Label'].append(inner_label)
    
    # Assign distances to each centroid.
    for j, target_label in enumerate(unique_labels):
        excel_data_full[f'Dist_to_{target_label}'].append(all_distances[idx, j])

# Convert to DataFrame and save to Excel.
excel_df_full = pd.DataFrame(excel_data_full)
excel_full_output_path = os.path.join(OUTPUT_DIR, 'all_data_with_tsne_and_centroid_distances.xlsx')
excel_df_full.to_excel(excel_full_output_path, index=False, engine='openpyxl')
print(f"✅ Excel file with full t-SNE results saved to: {excel_full_output_path}")

# --- 9. Generate Excel File with C_2 Sample Distances (Separate t-SNE) ---
# ------------------------------------------------------------------------------
print("\n--- Step 7: Generating Excel File with C_2 Sample Distances ---")

# Perform t-SNE on full C_2 data to get coordinates for distance calculations.
c2_indices_full = np.where(labels == 'C_2')[0]
if len(c2_indices_full) == 0:
    print("\nWarning: No C_2 samples found in the data. Skipping C_2 Excel output.")
else:
    print(f"--> Performing t-SNE on {len(c2_indices_full)} C_2 samples for distance calculations...")
    c2_features = feature_vectors[c2_indices_full]
    c2_standardized = scaler.fit_transform(c2_features)
    
    # Adjust perplexity for C_2 samples.
    c2_perplexity = min(30, max(5, len(c2_indices_full) // 3))
    print(f"Using perplexity={c2_perplexity} for C_2 t-SNE.")
    
    c2_tsne = TSNE(n_components=2, perplexity=c2_perplexity, random_state=42, init='pca', n_jobs=-1)
    c2_tsne_result = c2_tsne.fit_transform(c2_standardized)
    print(f"✅ C_2 t-SNE complete. Shape: {c2_tsne_result.shape}")

    # Create a DataFrame for the C_2 Excel file with distances to all centroids.
    excel_data_c2 = {
        'Filename': [],
        'Path': [],
        'tSNE_X': [],
        'tSNE_Y': [],
        'Dataset': [],
        'Inner_Label': [],
        'Dist_to_C_0': [],
        'Dist_to_C_1': [],
        'Dist_to_C_2': [],
        'Dist_to_C_3': [],
        'Dist_to_C_4': []
    }

    # Use vectorized distance calculation.
    c2_distances = cdist(c2_tsne_result, centroid_coords, metric='euclidean')
    
    for idx, orig_idx in enumerate(c2_indices_full):
        file_path = filenames[orig_idx]
        file_name = os.path.basename(file_path)
        inner_label = 'L_0' if 'L_0' in file_path else 'L_1' if 'L_1' in file_path else 'Unknown'
        
        excel_data_c2['Filename'].append(file_name)
        excel_data_c2['Path'].append(file_path)
        excel_data_c2['tSNE_X'].append(c2_tsne_result[idx, 0])
        excel_data_c2['tSNE_Y'].append(c2_tsne_result[idx, 1])
        excel_data_c2['Dataset'].append('C_2')
        excel_data_c2['Inner_Label'].append(inner_label)
        
        # Assign distances to each centroid.
        for j, target_label in enumerate(unique_labels):
            excel_data_c2[f'Dist_to_{target_label}'].append(c2_distances[idx, j])

    # Convert to DataFrame and save to Excel.
    excel_df_c2 = pd.DataFrame(excel_data_c2)
    excel_c2_output_path = os.path.join(OUTPUT_DIR, 'C2_data_with_all_centroids_ViT_tSNE.xlsx')
    excel_df_c2.to_excel(excel_c2_output_path, index=False, engine='openpyxl')
    print(f"✅ Excel file with C_2 sample distances saved to: {excel_c2_output_path}")

# --- 10. Visualization ---
# ------------------------------------------------------------------------------
print("\n--- Step 8: Visualizing t-SNE Results and Centroid Metrics ---")

# Scatter plot of t-SNE results with centroids (using subsampled points).
try:
    plt.style.use('ggplot')
except OSError:
    plt.style.use('default')
    plt.rcParams['grid.linestyle'] = '--'
    plt.rcParams['grid.linewidth'] = 0.5

fig, ax = plt.subplots(figsize=(14, 10))

# Plot subsampled t-SNE points.
palette = sns.color_palette("deep", n_colors=len(unique_labels))
color_map = dict(zip(unique_labels, palette))
for label, color in color_map.items():
    sub_label_mask = (sub_labels == label)
    ax.scatter(
        sub_tsne_result[sub_label_mask, 0],
        sub_tsne_result[sub_label_mask, 1],
        label=label,
        color=color,
        alpha=0.6,
        s=15
    )

# Plot centroids (from subsampled data).
for label, centroid in centroids.items():
    ax.scatter(
        centroid[0], centroid[1],
        color=color_map[label],
        marker='*',
        s=200,
        edgecolors='black',
        label=f'{label} Centroid'
    )

ax.set_title('2D t-SNE of ViT Feature Embeddings by Class with Centroids (Subsampled Points)', fontsize=18, fontweight='bold')
ax.set_xlabel('tSNE_X', fontsize=12)
ax.set_ylabel('tSNE_Y', fontsize=12)
ax.legend(title='Class ID', fontsize=10)
ax.grid(True, which='both', linestyle='--', linewidth=0.5)

tsne_plot_path = os.path.join(OUTPUT_DIR, 'tsne_visualization_with_centroids.png')
plt.savefig(tsne_plot_path, dpi=300, bbox_inches='tight')
print(f"✅ t-SNE visualization saved to: {tsne_plot_path}")

# Heatmap of Euclidean distances.
plt.figure(figsize=(10, 8))
sns.heatmap(euclidean_distance_df, annot=True, cmap='viridis', fmt='.2f', cbar_kws={'label': 'Euclidean Distance'})
plt.title('Pairwise Euclidean Distances Between Class Centroids in t-SNE Space', fontsize=16)
plt.xlabel('Class ID', fontsize=12)
plt.ylabel('Class ID', fontsize=12)

euclidean_heatmap_path = os.path.join(OUTPUT_DIR, 'centroid_euclidean_distances_heatmap.png')
plt.savefig(euclidean_heatmap_path, dpi=300, bbox_inches='tight')
print(f"✅ Euclidean distances heatmap saved to: {euclidean_heatmap_path}")

# Heatmap of cosine similarities.
plt.figure(figsize=(10, 8))
sns.heatmap(cosine_similarity_df, annot=True, cmap='viridis', fmt='.2f', cbar_kws={'label': 'Cosine Similarity'})
plt.title('Pairwise Cosine Similarities Between Class Centroids in t-SNE Space', fontsize=16)
plt.xlabel('Class ID', fontsize=12)
plt.ylabel('Class ID', fontsize=12)

cosine_heatmap_path = os.path.join(OUTPUT_DIR, 'centroid_cosine_similarities_heatmap.png')
plt.savefig(cosine_heatmap_path, dpi=300, bbox_inches='tight')
print(f"✅ Cosine similarities heatmap saved to: {cosine_heatmap_path}")

# Close all figures to free memory.
plt.close('all')

# --- 11. Save Processed Data for Future Use ---
# ------------------------------------------------------------------------------
print("\n--- Step 9: Saving t-SNE Results and Labels ---")

tsne_output_path = os.path.join(OUTPUT_DIR, 'tsne_result_full.npy')
np.save(tsne_output_path, tsne_result)
print(f"✅ Full t-SNE transformed data saved to: {tsne_output_path}")

labels_output_path = os.path.join(OUTPUT_DIR, 'labels_full.npy')
np.save(labels_output_path, labels)
print(f"✅ Full labels saved to: {labels_output_path}")

print("\n--- Script Finished Successfully ---")

In [ ]:
#build TSNE dataset 

# ==============================================================================
# SCRIPT: ViT_Feature_tSNE_Analysis_Full_C0_to_C5_NoCentroids.py
# DESCRIPTION: This script performs t-Distributed Stochastic Neighbor Embedding (t-SNE)
#              on the full set of Vision Transformer (ViT) feature vectors from all classes
#              (C_0 to C_5, 378,354 + C_5 samples) without subsampling. It generates an Excel
#              file with filename, full file path, t-SNE coordinates, dataset (C_0 to C_5),
#              and inner label (L_0 or L_1) for all samples. A scatter plot visualizes
#              subsampled points (1,000 total). Outputs are saved in a new folder. Compatible
#              with .pkl files containing (full_path, feature, dataset_label, inner_label)
#              tuples. Uses float16 to reduce memory usage and skips centroid distance
#              calculations to avoid MemoryError.
# AUTHOR:
# DATE: 2025-10-01
# ==============================================================================

# --- 1. Import Necessary Libraries ---
# ------------------------------------------------------------------------------
import numpy as np
import pickle
import os
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# --- 2. Configuration and Setup ---
# ------------------------------------------------------------------------------
# Define the base directory where feature files are located.
FEATURE_DIR = r"d:\.kaggle\new_FE_ViT_with_Path_and_Labels"

# Define the directory where all outputs (plots, data files) will be saved.
OUTPUT_DIR = r"d:\.kaggle\outputs_newRR5_vit_tsne_full"

# Create the output directory if it doesn't already exist.
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Subsampling configuration for visualization only
SUBSAMPLE_SIZE = 1000  # Maximum total samples for visualization
SUBSAMPLE_FRACTION = 0.1  # Fraction per class for visualization

print("--- Configuration Complete ---")
print(f"Feature source directory: {FEATURE_DIR}")
print(f"Output directory: {OUTPUT_DIR}\n")

# --- 3. Data Loading and Preprocessing ---
# ------------------------------------------------------------------------------
print("--- Step 1: Loading and Preparing Data ---")

# Initialize lists to aggregate data from all files.
labels = []
feature_vectors = []
filenames = []
dataset_labels = []
inner_labels = []

# Loop through the 6 classes (C_0 to C_5).
for i in range(6):
    hospital_id = f"C_{i}"
    
    # Construct the full path to the feature file.
    feature_file_path = os.path.join(FEATURE_DIR, f"2features_vit_{hospital_id}.pkl")

    # Check if the file exists before trying to open it.
    if os.path.exists(feature_file_path):
        print(f"--> Loading features for {hospital_id} from {feature_file_path}...")
        try:
            with open(feature_file_path, "rb") as f:
                # Load the data from the pickle file.
                data = pickle.load(f)
                
                # Validate .pkl file format
                if not data or not all(isinstance(item, tuple) and len(item) == 4 for item in data):
                    print(f"    ⚠️ Invalid format in {feature_file_path}. Expected tuples of (full_path, feature, dataset_label, inner_label).")
                    continue
                
                # Extract the feature vectors, filenames, dataset labels, and inner labels.
                class_features = [item[1] for item in data]
                class_filenames = [item[0] for item in data]
                class_datasets = [item[2] for item in data]
                class_inner_labels = [item[3] for item in data]
                
                # Validate dataset labels
                if not all(ds == hospital_id for ds in class_datasets):
                    print(f"    ⚠️ Warning: Some dataset labels in {feature_file_path} do not match {hospital_id}.")
                
                # Append to master lists.
                feature_vectors.extend(class_features)
                filenames.extend(class_filenames)
                dataset_labels.extend(class_datasets)
                inner_labels.extend(class_inner_labels)
                labels.extend([hospital_id] * len(class_features))
                
                print(f"    ✅ Loaded {len(class_features)} feature vectors for {hospital_id}.")
                # Report any 'Unknown' inner labels
                unknown_count = sum(1 for il in class_inner_labels if il == 'Unknown')
                if unknown_count > 0:
                    print(f"    ⚠️ Found {unknown_count} 'Unknown' inner labels in {hospital_id}.")
        except Exception as e:
            print(f"    ⚠️ Error loading {feature_file_path}: {e}. Skipping.")
    else:
        print(f"    ⚠️ File not found for {hospital_id} at {feature_file_path}. Skipping.")

# Convert to NumPy arrays with float16 to reduce memory usage
if not feature_vectors:
    print("\nError: No feature files were found. Exiting script.")
    exit()

try:
    feature_vectors = np.array(feature_vectors, dtype=np.float16)
    labels = np.array(labels)
    filenames = np.array(filenames)
    dataset_labels = np.array(dataset_labels)
    inner_labels = np.array(inner_labels)
except MemoryError as e:
    print(f"\nError: MemoryError during array conversion: {e}")
    print("Consider running on a system with 32+ GB RAM or a cloud instance (e.g., AWS EC2 r5.2xlarge with 64 GB RAM).")
    exit()

print(f"\nSuccessfully combined all features.")
print(f"Shape of the combined data matrix: {feature_vectors.shape}")

# --- 4. Data Standardization ---
# ------------------------------------------------------------------------------
print("\n--- Step 2: Standardizing Data ---")
scaler = StandardScaler()
try:
    standardized_data = scaler.fit_transform(feature_vectors)
except MemoryError as e:
    print(f"\nError: MemoryError during standardization: {e}")
    print("Consider running on a system with 32+ GB RAM or a cloud instance.")
    exit()
print("✅ Full data has been standardized (zero mean, unit variance).\n")

# --- 5. t-SNE Analysis on Full Data ---
# ------------------------------------------------------------------------------
print("--- Step 3: Applying t-SNE to Full Dataset ---")

# Adjust perplexity based on dataset size (lower to reduce memory)
n_samples = len(feature_vectors)
perplexity = min(10, max(5, n_samples // 3))  # Lower perplexity to reduce memory
print(f"Using perplexity={perplexity} for full t-SNE (adjusted for {n_samples} samples).")

# Initialize t-SNE with 2 components
try:
    tsne = TSNE(n_components=2, perplexity=perplexity, random_state=42, init='pca', n_jobs=-1)
    tsne_result = tsne.fit_transform(standardized_data)
except MemoryError as e:
    print(f"\nError: MemoryError during t-SNE computation: {e}")
    print(f"t-SNE on {n_samples} samples requires significant memory. Consider:")
    print("- Running on a system with 32+ GB RAM or a cloud instance (e.g., AWS EC2 r5.2xlarge).")
    print("- Using UMAP instead (contact for alternative script).")
    exit()

print("✅ t-SNE fitting and transformation complete.")
print(f"Shape of data after t-SNE: {tsne_result.shape}")
print(f"KL divergence (cost function): {tsne.kl_divergence_:.4f}")

# --- 6. Subsampling for Visualization ---
# ------------------------------------------------------------------------------
print("\n--- Step 4: Subsampling for Visualization ---")
sub_indices = []
num_classes = 5  # Updated to 6 classes (C_0 to C_4)
subsample_size_per_class = max(1, SUBSAMPLE_SIZE // num_classes)
unique_labels = sorted(set(labels))

for label in unique_labels:
    label_indices = np.where(labels == label)[0]
    n_label = len(label_indices)
    n_sub_label = min(n_label, max(subsample_size_per_class, int(n_label * SUBSAMPLE_FRACTION)))
    if n_label > 0:
        sub_label_idx = np.random.choice(label_indices, size=n_sub_label, replace=False)
        sub_indices.extend(sub_label_idx)

sub_indices = np.array(sub_indices)
sub_tsne_result = tsne_result[sub_indices]
sub_labels = labels[sub_indices]
sub_filenames = filenames[sub_indices]
print(f"    ✅ Subsampled {len(sub_indices)} points for visualization.")
print(f"    Subsampled t-SNE shape: {sub_tsne_result.shape}")

# --- 7. Generate Excel File with Full t-SNE Results ---
# ------------------------------------------------------------------------------
print("\n--- Step 5: Generating Excel File with Full t-SNE Results ---")

# Create a DataFrame for all samples' t-SNE coordinates and metadata
excel_data_full = {
    'Filename': [],
    'Path': [],
    'tSNE_X': [],
    'tSNE_Y': [],
    'Dataset': [],
    'Inner_Label': []
}

for idx in range(len(filenames)):
    excel_data_full['Filename'].append(os.path.basename(filenames[idx]))
    excel_data_full['Path'].append(filenames[idx])
    excel_data_full['tSNE_X'].append(tsne_result[idx, 0])
    excel_data_full['tSNE_Y'].append(tsne_result[idx, 1])
    excel_data_full['Dataset'].append(dataset_labels[idx])
    excel_data_full['Inner_Label'].append(inner_labels[idx])

# Convert to DataFrame and save to Excel
try:
    excel_df_full = pd.DataFrame(excel_data_full)
    excel_full_output_path = os.path.join(OUTPUT_DIR, 'all_data_with_tsne_C0_to_C5.xlsx')
    excel_df_full.to_excel(excel_full_output_path, index=False, engine='openpyxl')
    print(f"✅ Excel file with full t-SNE results saved to: {excel_full_output_path}")
except MemoryError as e:
    print(f"\nError: MemoryError during Excel generation: {e}")
    print("Consider running on a system with 32+ GB RAM or a cloud instance.")
    exit()

# --- 8. Visualization ---
# ------------------------------------------------------------------------------
print("\n--- Step 6: Visualizing t-SNE Results ---")

# Scatter plot of t-SNE results (subsampled points)
try:
    plt.style.use('ggplot')
except OSError:
    plt.style.use('default')
    plt.rcParams['grid.linestyle'] = '--'
    plt.rcParams['grid.linewidth'] = 0.5

fig, ax = plt.subplots(figsize=(14, 10))

# Plot subsampled t-SNE points
palette = sns.color_palette("deep", n_colors=len(unique_labels))
color_map = dict(zip(unique_labels, palette))
for label, color in color_map.items():
    sub_label_mask = (sub_labels == label)
    ax.scatter(
        sub_tsne_result[sub_label_mask, 0],
        sub_tsne_result[sub_label_mask, 1],
        label=label,
        color=color,
        alpha=0.6,
        s=15
    )

ax.set_title('2D t-SNE of ViT Feature Embeddings by Class (Subsampled, C_0 to C_5)', fontsize=18, fontweight='bold')
ax.set_xlabel('tSNE_X', fontsize=12)
ax.set_ylabel('tSNE_Y', fontsize=12)
ax.legend(title='Class ID', fontsize=10)
ax.grid(True, which='both', linestyle='--', linewidth=0.5)

tsne_plot_path = os.path.join(OUTPUT_DIR, 'tsne_visualization.png')
plt.savefig(tsne_plot_path, dpi=300, bbox_inches='tight')
print(f"✅ t-SNE visualization saved to: {tsne_plot_path}")

# Close all figures to free memory
plt.close('all')

# --- 9. Save Processed Data for Future Use ---
# ------------------------------------------------------------------------------
print("\n--- Step 7: Saving t-SNE Results and Labels ---")

tsne_output_path = os.path.join(OUTPUT_DIR, 'tsne_result_full.npy')
np.save(tsne_output_path, tsne_result)
print(f"✅ Full t-SNE transformed data saved to: {tsne_output_path}")

labels_output_path = os.path.join(OUTPUT_DIR, 'labels_full.npy')
np.save(labels_output_path, labels)
print(f"✅ Full labels saved to: {labels_output_path}")

print("\n--- Script Finished Successfully ---")